In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = 'cpu'

torch.manual_seed(777)

learning_rate = 0.5
batch_size = 10

In [3]:
mnist_train = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [4]:
W1 = nn.Parameter(torch.Tensor(784, 30)).to(device)
b1 = nn.Parameter(torch.Tensor(30)).to(device)
W2 = nn.Parameter(torch.Tensor(30, 10)).to(device)
b2 = nn.Parameter(torch.Tensor(10)).to(device)

In [5]:
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

Parameter containing:
tensor([ 0.3078, -1.9857,  1.0512,  1.5122, -1.0199, -0.7402, -1.3111,  0.6142,
        -0.6474,  0.1758], requires_grad=True)

In [6]:
def sigmoid(x):
    # sigmoid function
    return 1.0 / (1.0 + torch.exp(-x))
def sigmoid_prime(x):
    return  sigmoid(x) * (1 - sigmoid(x))

In [7]:
X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)[:1000]
Y_test = mnist_test.targets.to(device)[:1000]

In [8]:

i=0
while not i == 10000:
    for X, Y in data_loader:
        i += 1

        # forward
        X = X.view(-1, 28 * 28).to(device)
        Y = torch.zeros((batch_size, 10)).scatter_(1, torch.unsqueeze(Y, 1), 1).to(device)  # one-hot
        l1 = torch.add(torch.matmul(X, W1), b1)
        a1 = sigmoid(l1)
        l2 = torch.add(torch.matmul(a1, W2), b2)
        Y_pred = sigmoid(l2)

        diff = Y_pred - Y

        # Back prop (chain rule)
        d_l2 = diff * sigmoid_prime(l2)
        d_b2 = d_l2
        d_W2 = torch.matmul(torch.transpose(a1, 0, 1), d_l2)

        d_a1 = torch.matmul(d_l2, torch.transpose(W2, 0, 1))
        d_l1 = d_a1 * sigmoid_prime(l1)
        d_b1 = d_l1
        d_W1 = torch.matmul(torch.transpose(X, 0, 1), d_l1)

        W1 = W1 - learning_rate * d_W1
        b1 = b1 - learning_rate * torch.mean(d_b1, 0)
        W2 = W2 - learning_rate * d_W2
        b2 = b2 - learning_rate * torch.mean(d_b2, 0)

        if i % 1000 == 0:
            l1 = torch.add(torch.matmul(X_test, W1), b1)
            a1 = sigmoid(l1)
            l2 = torch.add(torch.matmul(a1, W2), b2)
            Y_pred = sigmoid(l2)
            acct_mat = torch.argmax(Y_pred, 1) == Y_test
            acct_res = acct_mat.sum()
            print(acct_res.item())
        if i == 10000:
            break

736
847
868
884
884
890
904
895
921
909
